# Consulta de Casas de Candomblé e Umbanda do Estado de São Paulo

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter as xl
import requests

In [2]:
print('pandas', pd.__version__)
print('numpy', np.__version__)

pandas 1.2.1
numpy 1.20.0


## Leitura da base da dados e ajustes iniciais

In [3]:
iles = pd.read_csv('data/base_iles_formatada.txt',
                   header=None,
                   sep='|',
                   error_bad_lines=False)
iles.columns = ['dirigente', 'casa_original', 'endereco_original']
iles

,dirigente,casa_original,endereco_original
0,"Ada de Omulu, Ialorixá (Adamaris Sá Oliveira)",Abaçá Oluayê Omode Okunrin Efan - candomblé q...,"Rua João Perestrello, 29 - Vila Mazzei - CEP..."
1,"Adão Conceição da Silva, Babalorixá",Ilê Axé Oxum Iyá Ayê - candomblé queto,"Rua Adolfo Lopes Rodrigues, 295 - Jardim Aloh..."
2,"Adejimi Aderotimi, Babalaô",Ilê Axé Olodumare - candomblé tradição,"Rua Dr. Jaci Barbosa, 375 - Vila Carrão - CEP..."
3,"Ademir Pereira Cunha, Pai",Tenda de Umbamba Mestre Junqueira e Maria do ...,"Rua dos Apiáres, 172-A - fundos - Jd. Santa T..."
4,"Adriano Luiz de Ogum, Pai",Templo de Umbanda Ogum Rompe Mato - umbanda,"Rua Alberto Andaló, 124 A - Freguesia do Ó - ..."
...,...,...,...
739,"Zezinho de Logum Edé, Babalorixá",Ebé Omó Logunbiyí - candomblé queto,"Rua João de Barro, 9 - Estância Montreal - Li..."
740,"Zilda de Vasconcelos Santos, Mãe",Tenda de Umbanda Cruzada Ogum e Mãe Maria do ...,"Rua André Mussoline, 44 - Jardim Eldorado - C..."
741,"Zilda Geraldo Pontes do Nascimento, Mãe",Abassá de Oxum e Ogum,"Rua Dr. Francisco José Longo, 299 - Saúde - C..."
742,"Zizi de Logum Edé, Babalorixá (Edson Alves da ...",Ilê Axé Araketu Awón Olodé - candomblé queto,"Rua Almada, 85 - Jardim Santo Alberto - CEP 0..."


Remoção limitação de exibição do conteúdo das células e aumento das linhas para 20 registros

In [4]:
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', None)

Remoção dos espaços em branco das colunas de texto

In [5]:
for column in iles.columns:
    if isinstance(iles[column].dtype, object):
        iles[column] = iles[column].str.strip()

Extração do CEP do endereço e remoção de caracteres inválidos do CEP

In [6]:
import string

iles['cep'] = iles['endereco_original'].str.extract(r'(\d{2}\.?\d{3}-? ?\d?\d?\d?)')
trans_table = ''.maketrans({key: '' for key in string.punctuation + string.whitespace})
iles['cep'] = iles['cep'].str.translate(trans_table)

Abertura do arquivo de CEPs previamente criado para o projeto,

**Se o arquivo não existir**, será criada uma lista única de CEPS e cada CEP será consultado no ViaCep para obtenção dos dados, em seguida o arquivo será salvo e também será disponibilizado no dataframe `ceps`

In [7]:
try:
    ceps = pd.read_parquet('data/ceps.parquet')
except:
    ceps_list = set(map(str, iles.cep.to_list()))
    
    enderecos_dict = {}

    for cep in ceps_list:
        url = f'https://viacep.com.br/ws/{cep}/json/'
        response = requests.get(f'{url}')
        enderecos_dict[cep] = response.json()
        
    pd.to_parquet('data/ceps.parquet')
    ceps = pd.DataFrame.from_dict(enderecos_dict, orient='index')

Alteração do campo `cep` para o index do dataframe para fazer o merge dos CEPs com o dataframe principal

In [8]:
ceps['cep'] = ceps.index
iles = iles.merge(ceps, how='left', on='cep')

Extração de email, ddd, telefone e url do campo de endereço

In [9]:
iles['email'] = iles['endereco_original'].str.extract(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)")
iles['url'] = iles['endereco_original'].str.extract(r"(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)")
iles['ddd_telefone1'] = iles['endereco_original'].str.extract(r"(\([1-9]{2}\))")
iles['ddd_telefone1'] = iles['ddd_telefone1'].str.translate(trans_table)
iles['telefone1'] = iles['endereco_original'].str.extract(r"((?:[2-8]|9[1-9])[0-9]{3}\-[0-9]{4})")
iles['telefone1'] = iles['telefone1'].str.translate(trans_table)

Separação do campo de endereço para extração do número de localização do logradouro

In [10]:
iles['log_num'] = iles['endereco_original'].str.split(" - ", expand=True)[0]
iles['numero_complemento'] = iles['log_num'].str.split(",").str[1].str.strip()

Separação do campo de casa para extração do nome do terreiro e da religião (na granularidade de especialização)

In [11]:
iles['terreiro'] = iles['casa_original'].str.split("- ", expand=True)[0].str.strip()
iles['especializacao'] = iles['casa_original'].str.split("- ", expand=True)[1].str.lower().str.strip()

Classificação das religiões em
 - candomblé
 - umbanda
 - tambor-de-mina
 - outras
 - não-declarada

In [12]:
def tipo_religiao(row):
    religioes = {'candomblé' : 'candomblé',
                 'umbanda': 'umbanda',
                 'tambor': 'tambor de mina'
                }

    if row['especializacao'] == "" or row['especializacao'] == None:
        return 'não-declarada'
    
    religiao_lista = []
    
    for key, value in religioes.items():
        if key in row['especializacao']:
            religiao_lista.append(value)
    
    if religiao_lista == []:
        religiao_lista.append('outras')
        
    return ', '.join(religiao_lista)


iles['religiao'] = iles.apply(tipo_religiao, axis=1)

In [13]:
iles.fillna("", inplace=True)

In [14]:
iles.rename(columns={'erro' : 'erro_cep'}, inplace=True)
iles = iles[['dirigente', 'terreiro', 'religiao', 'especializacao', 'logradouro', 'numero_complemento', 'bairro', 'localidade', 'uf', 'cep', 'erro_cep', 'ddd_telefone1', 'telefone1', 'email', 'url', 'casa_original', 'endereco_original']]
iles

,dirigente,terreiro,religiao,especializacao,logradouro,numero_complemento,bairro,localidade,uf,cep,erro_cep,ddd_telefone1,telefone1,email,url,casa_original,endereco_original
0,"Ada de Omulu, Ialorixá (Adamaris Sá Oliveira)",Abaçá Oluayê Omode Okunrin Efan,candomblé,candomblé queto-efã,Rua João Perestrello,29,Vila Nova Mazzei,São Paulo,SP,02314040,,11,62039930,,,Abaçá Oluayê Omode Okunrin Efan - candomblé queto-efã,"Rua João Perestrello, 29 - Vila Mazzei - CEP 02314-040 - São Paulo - SP telefone: (11) 6203-9930"
1,"Adão Conceição da Silva, Babalorixá",Ilê Axé Oxum Iyá Ayê,candomblé,candomblé queto,Rua Leopoldo Augusto Miguez,295,Melvi,Praia Grande,SP,11712170,,13,35944520,fenorixa@uol.com.br,,Ilê Axé Oxum Iyá Ayê - candomblé queto,"Rua Adolfo Lopes Rodrigues, 295 - Jardim Alohá - CEP 11712-170 - Praia Grande - SP telefone: (13) 3594-4520 / 9112-4300, e-mail: fenorixa@uol.com.br"
2,"Adejimi Aderotimi, Babalaô",Ilê Axé Olodumare,candomblé,candomblé tradição,Rua Doutor Jaci Barbosa,375,Vila Carrão,São Paulo,SP,03447000,,11,67812628,rotimi@terra.com.br,,Ilê Axé Olodumare - candomblé tradição,"Rua Dr. Jaci Barbosa, 375 - Vila Carrão - CEP 03447-000 - São Paulo - SP telefone: (11) 6781-2628, e-mail: rotimi@terra.com.br"
3,"Ademir Pereira Cunha, Pai",Tenda de Umbamba Mestre Junqueira e Maria do Coquinho,umbanda,umbanda,Rua dos Apiaris,172-A,Jardim Santa Terezinha (Pedreira),São Paulo,SP,04474140,,,,,,Tenda de Umbamba Mestre Junqueira e Maria do Coquinho - umbanda,"Rua dos Apiáres, 172-A - fundos - Jd. Santa Terezinha - CEP 04474-140 - São Paulo - SP"
4,"Adriano Luiz de Ogum, Pai",Templo de Umbanda Ogum Rompe Mato,umbanda,umbanda,Rua Alberto Andaló,124 A,Parque São Luís,São Paulo,SP,02840080,,11,39242371,,http://ogum.rompe.mato.sites.uol.com.br/index.html,Templo de Umbanda Ogum Rompe Mato - umbanda,"Rua Alberto Andaló, 124 A - Freguesia do Ó - CEP 02840-080 - São Paulo - SP telefone: (11) 3924-2371, e-mail: http://ogum.rompe.mato.sites.uol.com.br/index.html"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,"Zezinho de Logum Edé, Babalorixá",Ebé Omó Logunbiyí,candomblé,candomblé queto,Rua Apa,9,Campos Elíseos,São Paulo,SP,01201030,,19,,,,Ebé Omó Logunbiyí - candomblé queto,"Rua João de Barro, 9 - Estância Montreal - Limeira - SP / Correspondência para: Rua Apa, 304 apto 003 - CEP 01201-030 - São Paulo - SP - telefone (19) 3442 9343"
740,"Zilda de Vasconcelos Santos, Mãe",Tenda de Umbanda Cruzada Ogum e Mãe Maria do Rosário,umbanda,umbanda,Rua André Mussolino,44,Eldorado,Diadema,SP,09972140,,11,40594388,babazilda@yahoo.com.br,,Tenda de Umbanda Cruzada Ogum e Mãe Maria do Rosário - umbanda,"Rua André Mussoline, 44 - Jardim Eldorado - CEP 09972-140 - Diadema - SP - telefone: (11) 4059-4388 - e-mail: babazilda@yahoo.com.br"
741,"Zilda Geraldo Pontes do Nascimento, Mãe",Abassá de Oxum e Ogum,não-declarada,,Rua Doutor Francisco José Longo,299,Chácara Inglesa,São Paulo,SP,04140060,,,,,,Abassá de Oxum e Ogum,"Rua Dr. Francisco José Longo, 299 - Saúde - CEP 04140-060 - São Paulo - SP"
742,"Zizi de Logum Edé, Babalorixá (Edson Alves da Silva)",Ilê Axé Araketu Awón Olodé,candomblé,candomblé queto,Rua Almada,85,Jardim Santo Alberto,Santo André,SP,09260420,,11,49757975,,,Ilê Axé Araketu Awón Olodé - candomblé queto,"Rua Almada, 85 - Jardim Santo Alberto - CEP 09260-420 - Santo André - SP telefone: (11) 4975-7975 9294-4387"


In [15]:
def salvar_excel(dataframe, filename, sheetname='Planilha1', tablename='Tabela1'):
    """salvar_excel: salva o dataframe em uma tabela dentro de uma planilha no formato .xlsx"""
    
    rows, cols = dataframe.shape
    data = dataframe.to_dict('split')['data']
    headers = []
    
    try:
        for col in dataframe.columns:
            headers.append({'header' : col})

        wb = xl.Workbook(filename, {'nan_inf_to_errors': True})
        ws = wb.add_worksheet(sheetname)

        ws.add_table(0, 0, rows, cols - 1, {'name': tablename,
                                            'data': data,
                                            'columns': headers
        })
        print(f"Escrevendo arquivo:{filename}")
        wb.close()
        
        return True
    except:
        return False

    
print(salvar_excel.__doc__)

salvar_excel: salva o dataframe em uma tabela dentro de uma planilha no formato .xlsx


In [16]:
salvar_excel(iles, 'excel/iles.xlsx', 'Terreiros', 'Terreiro')

Escrevendo arquivo:excel/iles.xlsx


True

In [17]:
iles.groupby(['localidade'])[['dirigente']].count().sort_values('dirigente', ascending=False).head(15)

,dirigente
localidade,
São Paulo,340
Diadema,105
,98
São Bernardo do Campo,28
Guarulhos,15
Santo André,14
Osasco,12
Franca,9
Praia Grande,9


In [18]:
iles.groupby(['religiao'])[['dirigente']].count().sort_values('dirigente', ascending=False).head(15)

,dirigente
religiao,
candomblé,284
não-declarada,283
umbanda,150
"candomblé, umbanda",10
outras,9
tambor de mina,7
"candomblé, tambor de mina",1
